# IMDB 영화평 감성분석
-Pipeline
-TfidVectorizer + LogisticRegression

In [1]:
import numpy as np
import pandas as pd


In [2]:
df= pd.read_csv('labeledTrainData.tsv', sep='\t',quoting=3) #3:quote_none
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- 텍스트 전처리

In [3]:
# <br />태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')

In [4]:
import warnings
warnings.filterwarnings('ignore')
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()
df.review[0][:1000]

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

- Train/ test dataset분리

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

- Pipeline: TfidVectorizer + LogisticRegression

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [9]:
tvect = TfidfVectorizer(ngram_range=(1,2),stop_words='english')
lrc = LogisticRegression(random_state=2022)
pipeline = Pipeline([('TVECT',tvect), ('LR',lrc)])

In [10]:
# 학습
%time pipeline.fit(X_train,y_train)

CPU times: user 35.8 s, sys: 15 s, total: 50.8 s
Wall time: 34 s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('LR', LogisticRegression(random_state=2022))])

In [11]:
pipeline.score(X_test,y_test)

0.87472

- 최적 파라메터 찾기

In [12]:
from sklearn.model_selection import GridSearchCV
params = {
    'TVECT__max_df':[100,500],
    'LR__C':[1,10]
}

In [13]:
grid_pipe = GridSearchCV(pipeline,params,scoring='accuracy',cv=3, n_jobs=-1) #n-jobs=-1은 병행처리 되는 것
%time grid_pipe.fit(X_train,y_train)

CPU times: user 45.1 s, sys: 20.9 s, total: 1min 6s
Wall time: 3min 17s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TVECT',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'LR__C': [1, 10], 'TVECT__max_df': [100, 500]},
             scoring='accuracy')

In [14]:
grid_pipe.best_params_

{'LR__C': 10, 'TVECT__max_df': 500}

In [15]:
grid_pipe.best_estimator_.score(X_test,y_test)

0.87552

- 모델 저장하고 불러오기

In [16]:
import joblib
joblib.dump(grid_pipe.best_estimator_,'imdb_tvect_lr.pkl')

['imdb_tvect_lr.pkl']

In [17]:
!ls -l

total 103532
-rw-r--r-- 1 root root 72446571 Apr 28 04:33 imdb_tvect_lr.pkl
-rw-r--r-- 1 root root 33556378 Apr 28 04:06 labeledTrainData.tsv
drwxr-xr-x 1 root root     4096 Apr 25 13:46 sample_data


In [18]:
best_pipe = joblib.load('imdb_tvect_lr.pkl')

- 실제 데이터 적용

In [19]:
review = '''This is a movie made purely to satisfy the fans and there should be no doubt about that. 
No Way Home, in my opinion, is even better than Homecoming and Far From Home, 
and pretty much one of the best MCU movies of all time. 
It's a simple story, but the execution is fantastic. Even the smallest of surprises have a huge impact, 
and I could feel that in the theatre as I joined several other Spider-Man fans cheer out for both heroes and 
villains. The action sequences were brilliant; seeing them in 3D is totally worth the price of admission. 
Every actor delivered a believable, realistic performance, and especially our lead actor Tom Holland. 
The visual effects too were top notch and the editing was stupendous. 
Two and a half hours flew by real quick while watching this popcorn action entertainer. 
It won't be fair to reveal anything, so here I conclude my review, and recommend you to check out 
this new world of Spidey-ness on the big screen and in 3D. 
And once you've seen it, please don't spoil it for others, just like you won't want it spoiled for yourself.'''

In [20]:
# 텍스트 전처리
import re
review = re.sub('[^A-Za-z]',' ',review).strip()

In [21]:
best_pipe.predict([review])

array([1])